### Disclaimer

The following notebook was compiled for the course 'Geostatistics' at Ghent University (lecturer-in-charge: Prof. Dr. Ellen Van De Vijver; teaching assistant: Pablo De Weerdt). It consists of notebook snippets created by Michael Pyrcz. The code and markdown (text) snippets were edited specifically for this course, using the 'Jura data set' (Goovaerts, 1997) as example in the practical classes. Some new code snippets are also included to cover topics which were not found in the Geostastpy package demo books.

This notebook is for educational purposes.<br> 

Guidelines for getting started were adapted from the 'Environmental Soil Sensing' course at Ghent University (lecturer-in-charge: Prof. Dr. Philippe De Smedt).<br> 

The Jura data set was taken from: Goovaerts P., 1997. Geostatistics for Natural Resources Evaluation. Oxford University Press.

**Don't forget to save a copy on your Google drive before starting**

You can also 'mount' your Google Drive in Google colab to directly access your Drive folders (e.g. to access data, previous notebooks etc.)

Do not hesitate to contact us for questions or feel free to ask questions during the practical sessions.

# Geostatistics: Introduction to geostatistical data analysis with Python

In [ ]:
# Import required packages for setup
# -------------------------------------------- #

import sys
import os

In [ ]:
#  Clone the repository and add it to the path
if 'google.colab' in sys.modules:
    !git clone https://github.com/SENSE-UGent/E_I002454_Geostatistics.git
    sys.path.append('/content/E_I002454_Geostatistics') #Default location in Google Colab after cloning
else:
    # if you are not using Google Colab, change the path to the location of the repository
    sys.path.append(r'c:\Users\pdweerdt\Documents\Repos\E_I002454_Geostatistics')

# Import the setup function
from Utils.setup import check_and_install_packages

# Read the requirements.txt file
if 'google.colab' in sys.modules:
    requirements_path = '/content/E_I002454_Geostatistics/Utils/requirements.txt'
else:
    requirements_path = 'c:/Users/pdweerdt/Documents/Repos/E_I002454_Geostatistics/Utils/requirements.txt'

with open(requirements_path) as f:
    required_packages = f.read().splitlines()

# Check and install packages
check_and_install_packages(required_packages)

#### Load Required libraries

In [ ]:
import geostatspy
import geostatspy.GSLIB as GSLIB                              # GSLIB utilities, visualization and wrapper
import geostatspy.geostats as geostats                        # if this raises an error, you might have to check your numba isntallation   
print('GeostatsPy version: ' + str(geostatspy.__version__))   # these notebooks were tested with GeostatsPy version: 0.0.72

In [ ]:
# there was a small bug in the original kb2d_locations code from the geostatspy package
# we have fixed this bug in the Utils.func module

from Utils.func import kb2d_locations_v2

We will also need some standard packages. These should have been installed.

In [ ]:
from tqdm import tqdm                                         # suppress the status bar
from functools import partialmethod

tqdm.__init__ = partialmethod(tqdm.__init__, disable=True)
                                   
import numpy as np                                            # ndarrays for gridded data
                                       
import pandas as pd                                           # DataFrames for tabular data

import matplotlib.pyplot as plt                               # for plotting

from scipy import stats                                       # summary statistics

plt.rc('axes', axisbelow=True)                                # plot all grids below the plot elements

ignore_warnings = True                                        # ignore warnings?
if ignore_warnings == True:                                   
    import warnings
    warnings.filterwarnings('ignore')

from IPython.utils import io                                  # mute output from simulation

seed = 42                                                     # random number seed

### Optional libraries

These are not required to run the given version of this practical exercise, but might be useful if you want to extend this notebook with more code.

In [ ]:
#  import math library
import math

import cmath

In [ ]:
from scipy.stats import pearsonr                              # Pearson product moment correlation
from scipy.stats import spearmanr                             # spearman rank correlation    
                                   
import seaborn as sns                                         # advanced plotting

import matplotlib as mpl                                        

from matplotlib.ticker import (MultipleLocator, AutoMinorLocator) # control of axes ticks
from matplotlib.colors import ListedColormap 
import matplotlib.ticker as mtick 
import matplotlib.gridspec as gridspec

### Set the Working Directory

Do this to simplify subsequent reads and writes (avoid including the full address each time). 

##### For use in Google Colab

Run the following cell if you automatically want to get the data from the repository and store it on your Google Colab drive

In [ ]:
# change the working directory to the cloned repository

os.chdir('E_I002454_Geostatistics')

# get the current directory and store it as a variable

cd = os.getcwd()
print("Current Working Directory is " , cd)

##### For local use

Only run the following cell if you have the data locally stored.

In [ ]:
# set the working directory, place an r in front to address special characters
os.chdir(r'C:\\Users\\pdweerdt\\OneDrive - UGent\\I002454 - Geostatistics\\AY 2024-2025\\Practicals')

# get the current directory and store it as a variable

cd = os.getcwd()
print("Current Working Directory is " , cd)

### Loading Tabular & Gridded Data

Here's the section to load our data file into a Pandas' DataFrame object.

Let's load and visualize a grid also.

Check the datatype of your gridded data.

In this case it is actually also a .dat file, so we can use the same function to import it. The .grid extension was given to indicate that it is gridded data.

In [ ]:
# Here you can adjust the relative Path to the data folder

data_path = cd + '/Hard_data' 

In [ ]:
file_name = '//prediction.dat'

df = GSLIB.GSLIB2Dataframe(data_path + file_name) # read the data

df.head()

In [ ]:
grid_file_name = '//rocktype.grid'

# load the data

df_grid = GSLIB.GSLIB2Dataframe(data_path + grid_file_name)

df_grid.head()

### Define feature of interest

In [ ]:
feature = 'Cd'
unit = 'ppm'
dist_unit = 'km'

In [ ]:
#  define a colormap

cmap = plt.cm.inferno                                         # color map inferno

cmap_rainb = plt.cm.turbo # similar to what is shown on the slides

## Calculate some statistics

In P1 we calculated some statistics

In [ ]:
min_feat = round((df[feature].values).min(), 2)                    # calculate the minimum
max_feat = round((df[feature].values).max(), 2)                    # calculate the maximum
mean_feat = round((df[feature].values).mean(), 2)                  # calculate the mean
stdev_feat = round((df[feature].values).std(), 2)                  # calculate the standard deviation
n_feat = df[feature].values.size                                   # calculate the number of data

print('The minimum is ' + str(min_feat) + ' ' + str(unit) + '.')   # print univariate statistics
print('The maximum is ' + str(max_feat) + ' ' + str(unit) + '.')
print('The mean is ' + str(mean_feat) + ' ' + str(unit) + '.')
print('The standard deviation is ' + str(stdev_feat) + ' ' + str(unit) + '.')
print('The number of data is ' + str(n_feat) + '.')


# Ordinary Kriging for prediction Maps

Remember that we use both our prediction data and a variogram model as inputs into the OK prediction algorithm. We already loaded our prediction data and grid. Let's have a look at the grid and also define our search neigbourhood and the variogram model.


## Grid

We can initialise a new column into our grid dataframe for the OK prediction.

In [ ]:
df_grid[feature + 'OK'] = -99999 # assign a dummy value to the new feature

In [ ]:
xmin = 0; xmax = np.ceil(df.Xloc.max()) # range of x values
ymin = 0; ymax = np.ceil(df.Yloc.max()) # range of y values

In [ ]:
GSLIB.locmap_st(df_grid,'x', 'y', feature + 'OK',
                0, 5.2, ymin, ymax, 
                1, 5, # set the value range for the color map
                (
                    'Location Map Grid points ' 
               #   + str(grid_feature)
                 ), 
                 'X (km)', 'Y (km)',
             (
               #   str(grid_feature) + ' (' + str(unit) + ')'
                 ), 'gray')

plt.subplots_adjust(left=0.0, bottom=0.0, right=1.0, top=1.1, wspace=0.1, hspace=0.2)

We can hardly differentiate the individual grid points but they are surely there! Also ignore the colobar in this case as we focus on the grid locations.

## Variogram model

In [ ]:
nug = 0.3; nst = 1 # 2 nest structure variogram model parameters
it1 = 2;            # 1=spherical, 2=exponential, 3=gaussian
cc1 = 0.6; 
azi1 = 45; 
hmaj1 = 0.68; hmin1 = 0.68

if nst==2:

    it2 = 2; # prefereably same as it1
    cc2 = 4.2; # sill contribution of the second structure in major direction
    azi2 = 45; # direction with maximum spatial continuity (perpendicular to the major axis)
    hmaj2 = 1000; hmin2 = 1.1

else:

    it2= np.nan
    cc2= np.nan
    azi2= np.nan
    hmaj2= np.nan
    hmin2= np.nan

vario_mod = GSLIB.make_variogram(nug,nst,
                                it1,cc1,azi1,hmaj1,hmin1,
                                it2,cc2,azi2,hmaj2,hmin2
                                ) # make model object

## Search neighbourhood

In [ ]:
max_points = 15
min_points = 2
search_radii = [1, 1]   # search radius for neighbouring data

Let's try ordinary kriging

* to switch to ordinary kriging set the kriging ktype to 1

In [ ]:
%%capture --no-display    

tmin = -999; tmax = 9999

ktype = 1   # ordinary kriging
OK_kmap, OK_vmap = kb2d_locations_v2(df,"Xloc", "Yloc",feature,
                                        tmin, tmax, 
                                        df_grid, 'x', 'y',
                                        min_points, max_points, search_radii[0],
                                        ktype, None, vario_mod)

In [ ]:
# add the OK_kmap to the df_grid
df_grid[feature + 'OK'] = OK_kmap

# add tjhe OK_vmap to the df_grid
df_grid[feature + 'OK_var'] = OK_vmap

In [ ]:

cmap_rainb = plt.cm.turbo # similar to what is shown on the slides

plt.subplot(121)
GSLIB.locmap_st(df_grid,'x', 'y', feature + 'OK',
                0, 5.2, ymin, ymax, 
                0, 3, # set the value range for the color map
                ('Location Map ' + str(feature + 'OK ')), 'X (km)', 'Y (km)',
             (str(feature) + '_OK (' + str(unit) + ')'), cmap_rainb)

plt.subplots_adjust(left=0.0, bottom=0.0, right=1.0, top=1.1, wspace=0.1, hspace=0.2)

plt.subplot(122)
GSLIB.locmap_st(df_grid,'x', 'y', feature + 'OK_var',
                0, 5.2, ymin, ymax, 
                0, 3, # set the value range for the color map
                ('Location Map ' + str(feature + ' OK_var')), 'X (km)', 'Y (km)',
             (str(feature) + '_OK_var ()' + unit + '$^2$)' + ')'), cmap_rainb)

plt.subplots_adjust(left=0.0, bottom=0.0, right=2.0, top=1.0, wspace=0.3, hspace=0.3); plt.show()

## Jackknife validation

Repeat the process but choose validation locations as the grid where you want to make predictions...

In [ ]:
file_name = '//validation.dat'

df_val = GSLIB.GSLIB2Dataframe(data_path + file_name) # read the data

df_val.head()

In [ ]:
%%capture --no-display  

val_method = 'jk'

max_points = 15
min_points = 2
search_radii = [1,1]

feature = 'Cd'

# Initialize empty lists to add to the results df
val_method_vals = []

MPE_vals = []
MSPE_vals = []
RMSPE_vals = []
MAPE_vals = []
rel_nna_vals = []
Pr_vals = []
Sr_vals = []

results_df_v = pd.DataFrame()

# Perform validation initialize variables

a_c = 0 # for the cumulative error
a = 0 # for the error
a_c_a = 0 #for the absolute cum error
a_c_s = 0 #for the squared cum error

data_pred = df.copy()
data_val = df_val.copy()

# Perform OK
tmin = -999; tmax = 9999

ktype = 1   # ordinary kriging
OK_kmap, OK_vmap = kb2d_locations_v2(df,"Xloc", "Yloc",feature,
                                        tmin, tmax, 
                                        data_val, 'Xloc', 'Yloc',
                                        min_points, max_points, search_radii[0],
                                        ktype, None, 
                                        vario_mod # As modelled earlier!
                                        )

data_val['OK' + feature] = OK_kmap
data_val['OK' + feature + '_var'] = OK_vmap

# Calculate error on test set
data_val['r'] = data_val['OK' + feature] - data_val[feature] 

# print("r-value ", data_val['r'])

data_val['r_s'] = data_val['r']**2

data_val['r_a'] = data_val['r'].abs()

# Calculate average error
a = data_val['r'].mean()

a_s = data_val['r_s'].mean()

a_a = data_val['r_a'].mean()

a_c = data_val['r'].sum() #cumulative error

a_c_a = data_val['r_a'].sum() #cumulative absolute error

a_c_s = data_val['r_s'].sum() #cumulative squared error

# Round ac and aca
a_c = round(a_c, 2)
a_c_a = round(a_c_a, 2)
a_c_s = round(a_c_s, 2)

#calculate Mean prediction error
MPE = round(a_c/n_feat, 2)

print("Mean Prediction Error:", MPE)    

#Calculate Mean squared prediction error
MSPE = round(a_c_s/n_feat, 2)
print("Mean Squared Prediction Error:", MSPE)

#Calculate Root mean squared prediction error
RMSPE = round(math.sqrt(a_c_s/n_feat), 2)
print("Root Mean Squared Prediction Error:", RMSPE)

#calculate Mean absolute prediction error
MAPE = round(a_c_a/n_feat, 2)
print("Mean Absolute Prediction Error:", MAPE)

#Pearson correlation coefficient
#read in the data, drop na to avoid errors
data_cor = data_val.dropna(subset=[feature, 'OK' + feature])

#extract the columns of interest 
x = data_cor[feature]
y = data_cor['OK' + feature]

#calculate the Pearson's correlation coefficient 
corr_p, _ = pearsonr(x, y)
corr_p = round(corr_p, 2)
print('Pearsons correlation: %.3f' % corr_p)

# Spearman's Correlation:
#calculate the Spearman's correlation coefficient 
corr_s, _ = spearmanr(x, y)
corr_s = round(corr_s, 2)
print('Spearmans correlation: %.3f' % corr_s)

# Store the index values in the respective lists
MPE_vals.append(MPE)
MSPE_vals.append(MSPE)
RMSPE_vals.append(RMSPE)
MAPE_vals.append(MAPE)
Pr_vals.append(corr_p)
Sr_vals.append(corr_s)
val_method_vals.append(val_method)

# Create a new DataFrame to store the results for this variable and parameter settings
results_temp_df = pd.DataFrame()
results_temp_df['ValidationMethod'] = val_method_vals
results_temp_df['MPE'] = MPE_vals
results_temp_df['MSPE'] = MSPE_vals
results_temp_df['RMSPE'] = RMSPE_vals
results_temp_df['MAPE'] = MAPE_vals
results_temp_df['PearsonCorr'] = Pr_vals
results_temp_df['SpearmanCorr'] = Sr_vals

# Append the results for this variable and parameter settings to the main DataFrame
results_df_v_2d = pd.concat([results_df_v, results_temp_df], ignore_index=True)

results_df_v_2d.head()